In [3]:
import requests
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [5]:
requests.get("http://localhost:11434").content

b'Ollama is running'

In [9]:
from openai import OpenAI

In [13]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

client = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [17]:
from openai import OpenAI
from jinja2 import Template

def get_completion(prompt, model="llama3.2"):
    ''' Get a completion from the Ollama 
    Args:
        prompt (str): The prompt to send to the ollama
        model (str): The model to use for the completion
    Returns:
        str: The completion text
    '''
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

## Reusable Prompt Template

In [19]:
class PromptTemplate:
    ''' A class to represent a template for generating prompts with variables
    Attributes:
        template (str): The template string with variables
        input_variables (list): A list of the variable names in the template
    '''
    def __init__(self, template, input_variables):
        self.template = Template(template)
        self.input_variables = input_variables
    
    def format(self, **kwargs):
        return self.template.render(**kwargs)

# Simple template with one variable
simple_template = PromptTemplate(
    template="Provide a brief explanation of {{ topic }}.",
    input_variables=["topic"]
)

# More complex template with multiple variables
complex_template = PromptTemplate(
    template="Explain the concept of {{ concept }} in the field of {{ field }} to a {{ audience }} audience, concisely.",
    input_variables=["concept", "field", "audience"]
)

# Using the simple template
print("Simple Template Result:")
prompt = simple_template.format(topic="photosynthesis")
print(get_completion(prompt))

print("\n" + "-"*50 + "\n")

# Using the complex template
print("Complex Template Result:")
prompt = complex_template.format(
    concept="neural networks",
    field="artificial intelligence",
    audience="beginner"
)
print(get_completion(prompt))

Simple Template Result:
Photosynthesis is the process by which plants, algae, and some bacteria convert light energy from the sun into chemical energy in the form of organic compounds, such as glucose. This process occurs in specialized organelles called chloroplasts, which contain pigments like chlorophyll that absorb light energy.

The overall equation for photosynthesis is:

6 CO2 + 6 H2O + light energy → C6H12O6 (glucose) + 6 O2

In simpler terms, photosynthesis involves the following steps:

1. Light absorption: Chlorophyll and other pigments absorb light energy from the sun.
2. Water absorption: Plants absorb water from the soil through their roots.
3. Carbon dioxide absorption: Plants absorb carbon dioxide from the air through tiny openings on their leaves called stomata.
4. Energy conversion: The absorbed light energy is converted into chemical energy, which is stored in glucose.
5. Oxygen release: Oxygen is released as a byproduct of photosynthesis.

Photosynthesis is essentia

## Using Variables for Dynamic content

In [21]:
# Template with conditional content
conditional_template = PromptTemplate(
    template="My name is {{ name }} and I am {{ age }} years old. "
              "{% if profession %}I work as a {{ profession }}.{% else %}I am currently not employed.{% endif %} "
              "Can you give me career advice based on this information? answer concisely.",
    input_variables=["name", "age", "profession"]
)

# Using the conditional template
print("Conditional Template Result (with profession):")
prompt = conditional_template.format(
    name="Alex",
    age="28",
    profession="software developer"
)
print(get_completion(prompt))

print("\nConditional Template Result (without profession):")
prompt = conditional_template.format(
    name="Sam",
    age="22",
    profession=""
)
print(get_completion(prompt))

print("\n" + "-"*50 + "\n")


Conditional Template Result (with profession):
Hi Alex, 

As a 28-year-old software developer, here are some concise career advice suggestions:

1. **Continuously learn**: Stay updated with the latest technologies and programming languages to stay competitive.
2. **Specialize**: Focus on a specific area of software development (e.g., mobile app development, cloud computing) to increase your value.
3. **Networking**: Attend industry events, join online communities, and connect with peers to expand your professional network.
4. **Leadership skills**: Develop leadership skills by taking on projects, mentoring junior developers, or leading small teams.
5. **Career growth**: Consider transitioning into a technical lead, product manager, or entrepreneurship roles.

Remember, career growth is about continuous learning, self-improvement, and adapting to industry changes.

Best of luck!

Conditional Template Result (without profession):
Hi Sam, as a 22-year-old unemployed individual, here are s

In [25]:
# Template for list processing
list_template = PromptTemplate(
    template="Categorize these items into groups: {{ items }}. Provide the categories and the items in each category.",
    input_variables=["items"]
)

# Using the list template
print("List Template Result:")
prompt = list_template.format(
    items="apple, banana, carrot, hammer, screwdriver, pliers, novel, textbook, magazine"
)
print(get_completion(prompt))

List Template Result:
Here are the categorized groups:

**Group 1: Fruits**

* Apple
* Banana

**Group 2: Tools**

* Hammer
* Screwdriver
* Pliers

**Group 3: Books**

* Novel
* Textbook
* Magazine


## Advanced Template Technique

In [28]:
# Template with formatted list
list_format_template = PromptTemplate(
    template="Analyze the following list of items:\n"
              "{% for item in items.split(',') %}"
              "- {{ item.strip() }}\n"
              "{% endfor %}"
              "\nProvide a summary of the list and suggest any patterns or groupings.",
    input_variables=["items"]
)


# Using the formatted list template
print("Formatted List Template Result:")
prompt = list_format_template.format(
    items="Python, JavaScript, HTML, CSS, React, Django, Flask, Node.js"
)
print(get_completion(prompt))

print("\n" + "-"*50 + "\n")

Formatted List Template Result:
**Summary:**

The provided list consists of 8 programming languages, frameworks, and technologies used for web development. The list includes:

1. Python (a general-purpose programming language)
2. JavaScript (a scripting language used for client-side and server-side development)
3. HTML (a markup language for structuring content on the web)
4. CSS (a styling language for controlling layout and visual appearance of web pages)
5. React (a popular front-end library for building user interfaces)
6. Django (a high-level Python framework for building web applications)
7. Flask (a lightweight Python microframework for building web applications)
8. Node.js (a JavaScript runtime environment for building server-side applications)

**Patterns and Groupings:**

Based on the list, we can identify two main patterns:

1. **Front-end technologies:** HTML, CSS, React, and JavaScript are all related to client-side development, focusing on creating user interfaces and dyn